# Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn import utils
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import multiprocessing


In [2]:
from google.colab import drive
import glob

drive.mount('/content/drive')

Mounted at /content/drive


# Load and preprocess data

In [3]:
# CHANGE TO YOUR PATH
colab_resources_path = "/content/drive/My Drive/projectml"

In [4]:
data_files = glob.glob(colab_resources_path + "/*.csv")
data_files += glob.glob(colab_resources_path + "/*.py")
for data_file in data_files:
  print('Copying file {} to colab root.'.format(data_file))
  !cp "$data_file" .

Copying file /content/drive/My Drive/projectml/random.csv to colab root.
Copying file /content/drive/My Drive/projectml/nam.csv to colab root.
Copying file /content/drive/My Drive/projectml/am.csv to colab root.
Copying file /content/drive/My Drive/projectml/am_additional.csv to colab root.
Copying file /content/drive/My Drive/projectml/missclassified_roberta_mcc.csv to colab root.
Copying file /content/drive/My Drive/projectml/miss_5models.csv to colab root.
Copying file /content/drive/My Drive/projectml/missclassified_lr_robertas_ens_mcc.csv to colab root.
Copying file /content/drive/My Drive/projectml/data_preprocess.py to colab root.


In [5]:
from data_preprocess import getTrainData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Load full data

In [6]:
train = getTrainData(include_random=True, no_title=False, n_sentences=-1)

Count amount of words in data

In [7]:
train['text'].apply(lambda x: len(x.split(' '))).sum()

1207120

Function `tokenize_text` tokenizes texts, removes stopwords, punctuation, digits and single letter words.

Arguments:


*   `text` - text from train data

Returns:


*   list of `TaggedDocument` objects consisting of 2 parts: tokens and tags





In [8]:
stemmer=SnowballStemmer('english')
s=stopwords.words('english')
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        sent=re.sub('[^a-zA-Z]',' ',sent.lower())
        for word in nltk.word_tokenize(sent):
            if len(word) < 2 or word in s:
                continue
            tokens.append(stemmer.stem(word))
    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Apply `tokenize_text` to data 

In [ ]:
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.label]), axis=1)

Example of tokenized text

In [9]:
train_tagged[1]

TaggedDocument(words=['briberi', 'act', 'guidanc', 'detail', 'briberi', 'act', 'creat', 'new', 'offenc', 'section', 'commit', 'commerci', 'organis', 'fail', 'prevent', 'person', 'associ', 'bribe', 'anoth', 'person', 'behalf', 'organis', 'prove', 'adequ', 'procedur', 'place', 'prevent', 'person', 'associ', 'bribe', 'defenc', 'section', 'offenc', 'guidanc', 'publish', 'section', 'act', 'help', 'commerci', 'organis', 'size', 'sector', 'understand', 'sort', 'procedur', 'put', 'place', 'prevent', 'briberi', 'mention', 'section', 'quick', 'start', 'guid', 'also', 'publish', 'set', 'key', 'point', 'report', 'assess', 'impact', 'awar', 'briberi', 'act', 'among', 'small', 'medium', 'size', 'enterpris', 'export', 'good', 'oversea', 'also', 'avail', 'citi', 'london', 'polic', 'commiss', 'short', 'crime', 'prevent', 'video', 'design', 'use', 'compani', 'part', 'ongo', 'intern', 'train', 'staff', 'demonstr', 'impact', 'corrupt', 'investig', 'could', 'compani', 'individu', 'ignor', 'act', 'break', '

Split data on train and test

In [10]:
tr_tagged, ts_tagged = train_test_split(train_tagged, test_size=0.2, random_state=0)

# Model 1

In [12]:
cores = multiprocessing.cpu_count()

Create Doc2Vec model with distributed bag of words and train it

In [13]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 1e-6, workers=cores)
model_dbow.build_vocab([x for x in tqdm(tr_tagged.values)])

for epoch in range(20):
    model_dbow.train(utils.shuffle([x for x in tqdm(tr_tagged.values)]), total_examples=len(tr_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 1276/1276 [00:00<00:00, 1133403.62it/s]


Function `vec_for_learning` gets vectors from tokenized documents.

Arguments:


*   `model` - trained Doc2Vec model
*   `tagged_docs` - tokenized documents


Returns:


*   zip object of lists with labels and Doc2Vec vectors

In [11]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

Get train vectors from tokenized train and test data, train Logistic regression on train data and predict test data.

In [ ]:
y_train_1, x_train_1 = vec_for_learning(model_dbow, tr_tagged)
y_test_1, x_test_1 = vec_for_learning(model_dbow, ts_tagged)

lr1 = LogisticRegression(solver='liblinear', penalty='l2', C=15, max_iter=1000, random_state=0)

lr1.fit(x_train_1, y_train_1)
predictions = lr1.predict(x_test_1)

In [14]:
print('Accuracy score: ', accuracy_score(y_test_1, predictions))
print('Precision score: ', precision_score(y_test_1, predictions))
print('Recall score: ', recall_score(y_test_1, predictions))
print('F1 score: ',f1_score(y_test_1, predictions))
print('MCC score: ', matthews_corrcoef(y_test_1, predictions))


Accuracy score:  0.6959247648902821
Precision score:  0.7090909090909091
Recall score:  0.7048192771084337
F1 score:  0.7069486404833837
MCC score:  0.39100142800519605


# Model 2

Create Doc2Vec model with distributed memory and train it

In [15]:
model_dbow2 = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample = 1e-6, workers=cores)
model_dbow2.build_vocab([x for x in tqdm(tr_tagged.values)])

for epoch in range(20):
    model_dbow2.train(utils.shuffle([x for x in tqdm(tr_tagged.values)]), total_examples=len(tr_tagged.values), epochs=1)
    model_dbow2.alpha -= 0.002
    model_dbow2.min_alpha = model_dbow2.alpha

100%|██████████| 1276/1276 [00:00<00:00, 385086.48it/s]


CPU times: user 17.3 s, sys: 569 ms, total: 17.8 s
Wall time: 12.9 s


Get train vectors from tokenized train and test data, train Logistic regression on train data and predict test data.

In [16]:
y_train_2, x_train_2 = vec_for_learning(model_dbow2, tr_tagged)
y_test_2, x_test_2 = vec_for_learning(model_dbow2, ts_tagged)

lr2 = LogisticRegression(solver='liblinear', penalty='l2', C=15, max_iter=1000, random_state=0)

lr2.fit(x_train_2, y_train_2)
predictions = lr2.predict(x_test_2)

print('Accuracy score: ', accuracy_score(y_test_2, predictions))
print('Precision score: ', precision_score(y_test_2, predictions))
print('Recall score: ', recall_score(y_test_2, predictions))
print('F1 score: ',f1_score(y_test_2, predictions))
print('MCC score: ', matthews_corrcoef(y_test_2, predictions))


Accuracy score:  0.8683385579937304
Precision score:  0.8780487804878049
Recall score:  0.8674698795180723
F1 score:  0.8727272727272728
MCC score:  0.7364320419331226


# Model 3

Get train/test vectors by concatenating train/test vectors from Doc2Vec with distributed memory and Doc2Vec with distributed bag of words. Train Logistic regression on train data and predict test data.

1.   List item
2.   List item



In [17]:
x_train_3 = np.concatenate([x_train_1, x_train_2], axis=1)
x_test_3 =  np.concatenate([x_test_1, x_test_2], axis=1)

lr2 = LogisticRegression(solver='liblinear', penalty='l2', C=15, max_iter=1000, random_state=0)

lr2.fit(x_train_3, y_train_2)
predictions = lr2.predict(x_test_3)

print('Accuracy score: ', accuracy_score(y_test_2, predictions))
print('Precision score: ', precision_score(y_test_2, predictions))
print('Recall score: ', recall_score(y_test_2, predictions))
print('F1 score: ',f1_score(y_test_2, predictions))
print('MCC score: ', matthews_corrcoef(y_test_2, predictions))

Accuracy score:  0.8714733542319749
Precision score:  0.8980891719745223
Recall score:  0.8493975903614458
F1 score:  0.8730650154798762
MCC score:  0.744295118983762


# Cross-validation for model 2

Do 6-fold cross-validation with Doc2Vec model with distributed memory. 

In [41]:
n = 6
kf = KFold(n_splits=n, shuffle=True, random_state=0)

def cross_validation_for_2(kf_, model, X):

    i = 0

    accuracy = np.zeros(n)
    precision = np.zeros(n)
    recall = np.zeros(n)
    f1 = np.zeros(n)
    mcc = np.zeros(n)
    missclassified_idxs = []

    for train_index, test_index in kf_.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]

        train_tagged = X_train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r['label']]), axis=1)
        test_tagged = X_val.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r['label']]), axis=1)

        model_dbow = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample = 1e-6, workers=cores)
        model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

        for epoch in range(20):
            model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
            model_dbow.alpha -= 0.002
            model_dbow.min_alpha = model_dbow.alpha
            
        y_train, X_train_cv = vec_for_learning(model_dbow,train_tagged)
        y_val, X_test_cv = vec_for_learning(model_dbow,test_tagged)
        
        model.fit(X_train_cv, y_train)
        predictions = model.predict(X_test_cv)

        accuracy[i] = accuracy_score(y_val, predictions)
        precision[i] = precision_score(y_val, predictions)
        recall[i] = recall_score(y_val, predictions)
        f1[i] = f1_score(y_val, predictions)
        mcc[i] = matthews_corrcoef(y_val, predictions)
        
        i += 1
    return np.mean(accuracy), np.mean(precision), np.mean(recall), np.mean(f1), np.mean(mcc), missclassified_idxs

In [20]:
lr = LogisticRegression(solver='liblinear', penalty='l2', C=15, max_iter=1000, random_state=0)
accuracy, precision, recall, f1, mcc, missclassified_idxs = cross_validation_for_2(kf, lr, train)

100%|██████████| 1329/1329 [00:00<00:00, 2155541.38it/s]


Fold index:  0
Accuracy score:  0.868421052631579
Precision score:  0.855072463768116
Recall score:  0.8872180451127819
F1 score:  0.8708487084870848
MCC score:  0.7373633501503364


100%|██████████| 1329/1329 [00:00<00:00, 363781.90it/s]


Fold index:  1
Accuracy score:  0.8759398496240601
Precision score:  0.9230769230769231
Recall score:  0.8391608391608392
F1 score:  0.8791208791208791
MCC score:  0.7559071349320328


100%|██████████| 1329/1329 [00:00<00:00, 1338029.29it/s]


...

Fold index:  3
Accuracy score:  0.849624060150376
Precision score:  0.8134328358208955
Recall score:  0.8790322580645161
F1 score:  0.8449612403100775
MCC score:  0.7013843945170944


100%|██████████| 1329/1329 [00:00<00:00, 563451.94it/s]


Fold index:  4
Accuracy score:  0.8796992481203008
Precision score:  0.8992248062015504
Recall score:  0.8592592592592593
F1 score:  0.8787878787878787
MCC score:  0.7602806026407116


100%|██████████| 1330/1330 [00:00<00:00, 519841.98it/s]


Fold index:  5
Accuracy score:  0.8566037735849057
Precision score:  0.8560606060606061
Recall score:  0.8560606060606061
F1 score:  0.8560606060606061
MCC score:  0.7132034632034632


In [21]:
print('Accuracy score: ', accuracy)
print('Precision score: ', precision)
print('Recall score: ', recall)
print('F1 score: ', f1)
print('MCC score: ', mcc)      

Accuracy score:  0.8620655412115195
Precision score:  0.8643806155537086
Recall score:  0.8620971432516424
F1 score:  0.8625612580101074
MCC score:  0.7253806692786666


# Cross-validation for model 3


Do 6-fold cross-validation with concatenated vectors from Doc2Vec model with distributed memory and vactors from Doc2Vec model with distributed bag of words.

In [37]:
n = 6
kf = KFold(n_splits=n, shuffle=True, random_state=0)
def cross_validation_for_3(kf_, model, X):
    i = 0

    accuracy = np.zeros(n)
    precision = np.zeros(n)
    recall = np.zeros(n)
    f1 = np.zeros(n)
    mcc = np.zeros(n)
    missclassified_idxs = []

    for train_index, test_index in kf_.split(X):

        X_train, X_val = X.iloc[train_index], X.iloc[test_index]

        train_tagged = X_train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r['label']]), axis=1)
        test_tagged = X_val.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r['label']]), axis=1)

        model_dbow = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample = 1e-6, workers=cores)
        model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

        model_dbow2 = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 1e-6, workers=cores)        
        model_dbow2.build_vocab([x for x in tqdm(train_tagged.values)])

        for epoch in range(20):

            model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
            model_dbow.alpha -= 0.002
            model_dbow.min_alpha = model_dbow.alpha

            model_dbow2.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
            model_dbow2.alpha -= 0.002
            model_dbow2.min_alpha = model_dbow.alpha

        y_train1, X_train_cv1 = vec_for_learning(model_dbow,train_tagged)
        y_val, X_test_cv1 = vec_for_learning(model_dbow,test_tagged)
        y_train2, X_train_cv2 = vec_for_learning(model_dbow2,train_tagged)
        y_val, X_test_cv2 = vec_for_learning(model_dbow2,test_tagged)
        
        X_train_cv = np.concatenate([X_train_cv1, X_train_cv2], axis=1)
        X_test_cv =  np.concatenate([X_test_cv1, X_test_cv2], axis=1)

        model.fit(X_train_cv, y_train1)
        predictions = model.predict(X_test_cv)

        accuracy[i] = accuracy_score(y_val, predictions)
        precision[i] = precision_score(y_val, predictions)
        recall[i] = recall_score(y_val, predictions)
        f1[i] = f1_score(y_val, predictions)
        mcc[i] = matthews_corrcoef(y_val, predictions)
        
        i += 1
    return np.mean(accuracy), np.mean(precision), np.mean(recall), np.mean(f1), np.mean(mcc), missclassified_idxs

In [38]:
lr = LogisticRegression(solver='liblinear', penalty='l2', C=15, max_iter=1000, random_state=0)
accuracy, precision, recall, f1, mcc, missclassified_idxs = cross_validation_for_3(kf, lr, train)

100%|██████████| 1330/1330 [00:00<00:00, 1362918.23it/s]


In [39]:
print('Accuracy score: ', accuracy)
print('Precision score: ', precision)
print('Recall score: ', recall)
print('F1 score: ', f1)
print('MCC score: ', mcc)      

Accuracy score:  0.8677211897668701
Precision score:  0.8838972274094824
Recall score:  0.8493790511192888
F1 score:  0.8657170722133856
MCC score:  0.7368339325852231


# Hyperparameter search

This could be used to tune hyperparameters of Logistic regression, but we decided not to tune hyperparameters, because


1.   Tuning logistic regression will not increase accuracy significantly
2.   We are not going to use Doc2Vec + LR for ensemble
3.   It takes long time to compute



In [ ]:
C_range = np.arange(10, 22)

# liblinear
penalty=['none', 'l1', 'l2']
intercept_scaling = np.arange(0.8, 1.3, 0.1)
verbose = np.arange(0, 6)

# lbfgs
penalty=['none','l2']
verbose = np.arange(0, 6)

# saga
penalty=['none', 'l1', 'l2', 'elasticnet']
verbose = np.arange(0, 6)

solvers = {}
solvers['liblinear'] = {
    'penalty': ['_l1', 'l2'],
    'intercept_scaling': np.arange(0.8, 1.3, 0.1),
    'verbose': np.arange(0, 6)
}

solvers['lbfgs'] = {
    'verbose': np.arange(0, 6)
}

solvers['saga'] = {
    'penalty': ['none', 'l1', 'l2', 'elasticnet'],
    'l1_ratio': np.arange(0, 0.2, 0.02)
}

solvers['saga'] = {
    'penalty': ['elasticnet'],
    'l1_ratio': np.arange(0, 0.2, 0.02)
}

    
# Liblinear
results_array = []
mg = np.array(np.meshgrid(solvers['lbfgs']['verbose'], C_range)).T.reshape(-1,2)

for params in mg:
    verbose=int(params[0])
    C=int(params[1])

    name = f'saga-{verbose}-{C}'
    print(name)

    lr = LogisticRegression(solver='lbfgs', verbose=verbose, penalty='l2', C=C, max_iter=1000, multi_class='ovr', random_state=0)
    
    accuracy, precision, recall, f1, mcc, missclassified_idxs = cross_validation_for_2(kf, lr, train)
    
    results_array.append([name , accuracy, precision, recall, f1, mcc])
    print([name , accuracy, precision, recall, f1, mcc])
results = pd.DataFrame(results_array, columns = ['model', 'accuracy', 'precision', 'recall', 'f1', 'mcc'])
results

saga-0-10


100%|██████████| 1330/1330 [00:00<00:00, 1907153.61it/s]


['saga-0-10', 0.8601976639712489, 0.8698672112030824, 0.8492918413829562, 0.859030175336688, 0.7211909929230961]
saga-0-11


100%|██████████| 1330/1330 [00:00<00:00, 1426706.99it/s]


['saga-0-11', 0.8620773632193691, 0.8612027153437785, 0.8636540977938828, 0.862336117335424, 0.7240478868027552]
saga-0-12


...

100%|██████████| 1329/1329 [00:00<00:00, 329816.58it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1329/1329 [00:00<00:00, 275829.09it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
100%|██████████| 1329/1329 [00:00<00:00, 810797.09it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1329/1329 [00:00<00:00, 612149.13it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[P

['saga-3-16', 0.8545443798174682, 0.8615166687033188, 0.8480508147033333, 0.8540242779594568, 0.7101609504819996]
saga-3-17


100%|██████████| 1329/1329 [00:00<00:00, 430508.96it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
100%|██████████| 1329/1329 [00:00<00:00, 1284680.81it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1329/1329 [00:00<00:00, 580528.02it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1329/1329 [00:00<00:00, 1109299.51it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


['saga-3-17', 0.8501678725114674, 0.8473624657289162, 0.8596559307510073, 0.8522760203361291, 0.7021849930934025]
saga-3-18


100%|██████████| 1329/1329 [00:00<00:00, 324649.39it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1329/1329 [00:00<00:00, 1061758.10it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
100%|██████████| 1329/1329 [00:00<00:00, 1886372.26it/s]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
  0%|          | 0/1329 [00:00<?, ?it/s]

In [ ]:
results[results['mcc '] >= 0.80]

# Conclusion

Accuracy and MCC score are lower than accuracy and MCC of RoBERTa and logistic regression, therefore we decided not to use Doc2Vec in the ensembles.